In [1]:

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
# import geopandas as gpd
# import matplotlib.pyplot as plt

# # Define the paths
# input_file_path = "./antarctic_index.gpkg"
# map_path = "/Users/nathanbekele/Downloads/Quantarctica3/Miscellaneous/SimpleBasemap/ADD_DerivedLowresBasemap.shp"

# # Read the GeoPackage file
# gdf = gpd.read_file(input_file_path)

# # Print the first few rows of the GeoDataFrame
# print("First few rows of the GeoDataFrame:")
# print(gdf.head())

# # Display unique institutions to check if 'AWI' is present
# print("\nUnique institutions in the dataset:")
# print(gdf['institution'].unique())

# # Filter the GeoDataFrame for the institution 'AWI'
# filtered_gdf = gdf[gdf['institution'] == 'AWI']

# # Print the unique institutions in the filtered GeoDataFrame
# print("\nUnique institutions in the filtered GeoDataFrame:")
# print(filtered_gdf['institution'].unique())

# # Check if the filtered GeoDataFrame is empty
# print(f"\nFiltered GeoDataFrame for AWI is empty: {filtered_gdf.empty}")

# # Print the first few rows of the filtered GeoDataFrame to confirm filtering
# print("\nFirst few rows of the filtered GeoDataFrame:")
# print(filtered_gdf.head())

# # Print the CRS of the filtered GeoDataFrame
# print("\nCRS of the filtered GeoDataFrame before transformation:")
# print(filtered_gdf.crs)

# # Read the basemap shapefile
# basemap = gpd.read_file(map_path)

# # Print the CRS of the basemap
# print("\nCRS of the basemap:")
# print(basemap.crs)

# # Ensure the CRS matches between the basemap and the GeoDataFrame
# if not filtered_gdf.empty:
#     filtered_gdf = filtered_gdf.to_crs(basemap.crs)

# # Print the CRS of the filtered GeoDataFrame after transformation
# print("\nCRS of the filtered GeoDataFrame after transformation:")
# print(filtered_gdf.crs)

# # Function to plot data on the basemap
# def plot_data_on_basemap(basemap, gdf):
#     fig, ax = plt.subplots(figsize=(10, 10))

#     # Set background color to light blue
#     fig.patch.set_facecolor('lightblue')
#     ax.set_facecolor('lightblue')
    
#     # Plot basemap with Antarctica as white
#     basemap.plot(ax=ax, color='white', edgecolor='black')
    
#     # Plot the data
#     if not gdf.empty:
#         print("\nPlotting data:")
#         print(gdf)
#         gdf.plot(ax=ax, color='red', linewidth=2)
#     else:
#         print("GeoDataFrame is empty, nothing to plot.")

#     # Set limits to zoom in on Antarctica
#     ax.set_xlim(-3e6, 3e6)
#     ax.set_ylim(-3e6, 3e6)
    
#     # Remove axis for cleaner look
#     ax.set_axis_off()
    
#     plt.title('Antarctica Basemap with AWI Data Overlay')
#     plt.show()

# # Plot the data
# plot_data_on_basemap(basemap, filtered_gdf)


In [3]:
print(basemap.columns)

NameError: name 'basemap' is not defined

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import os
from matplotlib_scalebar.scalebar import ScaleBar

# Define the paths
input_file_path = "./antarctic_index.gpkg"
map_path = "/Users/nathanbekele/Downloads/Quantarctica3/Miscellaneous/SimpleBasemap/ADD_DerivedLowresBasemap.shp"
output_folder = "./data3"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# List all layers in the GeoPackage
try:
    layers = gpd.io.file.fiona.listlayers(input_file_path)
    if layers is None:
        raise ValueError("No layers found in the GeoPackage.")
    print("Layers in the GeoPackage:")
    print(layers)
except Exception as e:
    print(f"Error listing layers in the GeoPackage: {e}")
    layers = []

if not layers:
    print("No layers found or error in listing layers. Exiting.")
else:
    # Create a dictionary to map institutions to layers
    institution_layers = {}

    # Populate the dictionary
    for layer in layers:
        try:
            gdf_head = gpd.read_file(input_file_path, layer=layer, rows=10)  # Read first 10 rows
            gdf_tail = gpd.read_file(input_file_path, layer=layer, skiprows=lambda x: x < (gdf_head.shape[0] - 10))  # Read last 10 rows
            if 'institution' in gdf_head.columns:
                institutions = set(gdf_head['institution'].unique()).union(set(gdf_tail['institution'].unique()))
                for institution in institutions:
                    if institution is not None:
                        if institution not in institution_layers:
                            institution_layers[institution] = []
                        institution_layers[institution].append(layer)
        except Exception as e:
            print(f"Error reading layer {layer}: {e}")

    print("Institution layers mapping:")
    print(institution_layers)

    # Read the basemap shapefile
    basemap = gpd.read_file(map_path)

    # Function to plot data on the basemap with color-coding based on availability
    def plot_data_on_basemap(basemap, gdf, institution, output_folder):
        fig, ax = plt.subplots(figsize=(10, 10))

        # Set background color to white
        fig.patch.set_facecolor('white')
        ax.set_facecolor('white')
        
        # Plot basemap with Antarctica as white
        basemap.plot(ax=ax, color='white', edgecolor='black')
        
        # Define colors based on availability
        colors = {'u': 'darkblue', 's': 'red', '': 'darkgrey'}
        labels = {'u': 'Available', 's': 'Unavailable', '': 'Dark Grey'}
        
        # Plot the data for the institution
        if not gdf.empty:
            for availability in gdf['availability'].unique():
                subset = gdf[gdf['availability'] == availability]
                color = colors.get(availability, 'darkgrey')
                subset.plot(ax=ax, color=color, markersize=0.2, linewidth=0.2, label=f"{labels.get(availability, 'Dark Grey')}")
        else:
            print("GeoDataFrame is empty, nothing to plot.")

        # Set limits to zoom in on Antarctica
        ax.set_xlim(-3e6, 3e6)
        ax.set_ylim(-3e6, 3e6)
        
        # Add axis labels for coordinates
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        
        # Add a scalebar
        scalebar = ScaleBar(1, location='lower right')
        ax.add_artist(scalebar)
        
        plt.title(f'Antarctica Basemap with {institution} Data Overlay')
        plt.legend()
        
        # Save the plot to the data folder
        output_path = os.path.join(output_folder, f'Antarctica_coverage_{institution}.png')
        plt.savefig(output_path)
        plt.close(fig)
        print(f"Saved map for {institution} to {output_path}")

    # Iterate through each institution and create maps
    all_data = []
    for institution, layers in institution_layers.items():
        institution_data = []

        print(f"\nProcessing institution: {institution}")
        
        for layer in layers:
            print(f"Checking layer: {layer}")
            try:
                gdf = gpd.read_file(input_file_path, layer=layer)  # Read the full layer
                
                if 'institution' in gdf.columns and institution in gdf['institution'].unique():
                    print(f"Layer '{layer}' contains '{institution}'.")
                    institution_data.append(gdf[gdf['institution'] == institution])
                else:
                    print(f"Layer '{layer}' does not contain '{institution}' or does not have 'institution' column.")
            except Exception as e:
                print(f"Error processing layer {layer}: {e}")

        # Combine all dataframes for the institution
        if institution_data:
            institution_gdf = gpd.GeoDataFrame(pd.concat(institution_data, ignore_index=True))
            all_data.append(institution_gdf)
        else:
            institution_gdf = gpd.GeoDataFrame()

        # Ensure the CRS matches between the basemap and the GeoDataFrame
        if not institution_gdf.empty:
            institution_gdf = institution_gdf.to_crs(basemap.crs)

        # Plot and save the aggregated data for the institution
        plot_data_on_basemap(basemap, institution_gdf, institution, output_folder)

    # Generate an overview map
    if all_data:
        all_data_gdf = gpd.GeoDataFrame(pd.concat(all_data, ignore_index=True))
        plot_data_on_basemap(basemap, all_data_gdf, 'Overview', output_folder)

    print("All maps have been created and saved to the data3 folder.")
